## Embeddings proces:

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv(r"D:\Afra Zaib\Semester 6th\ARS\TwoStageMovieRecommender\MovieRecommender\DataPreprocess\cleaned_movies.csv")


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text, tokenizer, bert_model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    # Use the output embeddings of the [CLS] token (first token)
    return outputs.last_hidden_state[0][0].numpy()


df['overview_embeddings'] = df['Overview'].apply(lambda x: get_bert_embeddings(x, tokenizer, bert_model))
df['plot_keywords_embeddings'] = df['Plot Kyeword'].apply(lambda x: get_bert_embeddings(x, tokenizer, bert_model))


label_encoder = LabelEncoder()
df['genre_encoded'] = label_encoder.fit_transform(df['Generes'])


embedding_dim = 50  
genre_embeddings = torch.nn.Embedding(len(label_encoder.classes_), embedding_dim)


df['genre_embeddings'] = df['genre_encoded'].apply(lambda x: genre_embeddings(torch.tensor(x)).detach().numpy())


def combine_embeddings(row):
    combined = np.concatenate([
        row['overview_embeddings'],
        row['plot_keywords_embeddings'],
        row['genre_embeddings']
    ])
    return combined

df['combined_embeddings'] = df.apply(combine_embeddings, axis=1)


sample_embeddings = df['combined_embeddings'].head(3).tolist()
for i, vec in enumerate(sample_embeddings):
    print(f"Sample {i+1} embedding vector: {vec[:10]}...")  
